In [1]:
!huggingface-cli whoami

m-aliabbas1


In [2]:
from huggingface_hub import HfApi

user_id = HfApi().whoami()["name"]


In [6]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ['text_file.txt']


CPU times: user 27 µs, sys: 0 ns, total: 27 µs
Wall time: 33.4 µs


In [7]:
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [8]:
!mkdir Waspak_papi_BERTo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
path  = 'Waspak_papi_BERTo'

In [10]:
tokenizer.save_model(path)

['Waspak_papi_BERTo/vocab.json', 'Waspak_papi_BERTo/merges.txt']

In [11]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    f"./{path}/vocab.json",
    f"./{path}/merges.txt",
)

In [12]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [14]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [15]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(f"./{path}", max_len=512)

In [16]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [18]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
exp_bert_pretraining.ipynb	 Tokenizer.ipynb
exp_data_cleaning_and_eda.ipynb  tokenizer_roberta_papia
exp_roberta.ipynb		 tokenizer_tiny_bert_papia
exp_tiny_bert_tokenizer.ipynb	 Untitled.ipynb
models				 WaspakBERTo
test_text_file.txt		 WaspakBERTo1
text_col.csv			 Waspak_papi_BERTo
text_file.txt


In [19]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./text_file.txt",
    block_size=128,
)

CPU times: user 4min 44s, sys: 2.67 s, total: 4min 47s
Wall time: 14.9 s


In [20]:
test_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./test_text_file.txt",
    block_size=128,
)

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [26]:
training_args = TrainingArguments(
    output_dir=f'{path}/',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=40, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=30,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=30,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

In [27]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=test_dataset
    
)

In [28]:
%%time
trainer.train()

Step,Training Loss,Validation Loss
30,8.501200,8.010108
60,7.842200,7.680418


/home/mohammad/miniconda3/envs/ali_nlp/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/mohammad/miniconda3/envs/ali_nlp/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


CPU times: user 8min 31s, sys: 1min 39s, total: 10min 11s
Wall time: 7min 25s


TrainOutput(global_step=70, training_loss=8.101715741838728, metrics={'train_runtime': 444.9074, 'train_samples_per_second': 104.629, 'train_steps_per_second': 0.157, 'total_flos': 1466027210219520.0, 'train_loss': 8.101715741838728, 'epoch': 9.49})

In [30]:
trainer.save_model(f"./{path}")

In [32]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=f"./{path}",
    tokenizer=f"./{path}"
)

In [33]:
model.push_to_hub('roberta-waspak-2023-papia1')

pytorch_model.bin:   0%|          | 0.00/334M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/m-aliabbas1/roberta-waspak-2023-papia1/commit/a45ba697d3e340fe052b1644eed672fb0aea1c93', commit_message='Upload RobertaForMaskedLM', commit_description='', oid='a45ba697d3e340fe052b1644eed672fb0aea1c93', pr_url=None, pr_revision=None, pr_num=None)

In [34]:
tokenizer.push_to_hub('roberta-waspak-2023-papia1')

CommitInfo(commit_url='https://huggingface.co/m-aliabbas1/roberta-waspak-2023-papia1/commit/8c7664d316d58b0e039d2bd6c4927a3abc8843f6', commit_message='Upload tokenizer', commit_description='', oid='8c7664d316d58b0e039d2bd6c4927a3abc8843f6', pr_url=None, pr_revision=None, pr_num=None)